In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import cv2
import zipfile
import os
from matplotlib import pyplot as plt
import random
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import matplotlib.image as mpimg

from tensorflow.keras.optimizers import RMSprop, Adam
from imblearn.over_sampling import SMOTE
from keras.models import Model

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_set = '/content/drive/MyDrive/fruit_classification/data/train/train/'
train_set

'/content/drive/MyDrive/fruit_classification/data/train/train/'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   horizontal_flip=True,
                                   shear_range = 0.3,validation_split=0.2,
                                   zoom_range = 0.3)

In [ ]:
train_data = train_datagen.flow_from_directory(train_set,
                                              target_size = (100, 100),
                                              class_mode = 'categorical', batch_size = 64, subset="training")

valid_data = train_datagen.flow_from_directory(train_set,
                                              target_size = (100, 100),
                                              class_mode = 'categorical', batch_size = 64, subset="validation")

Found 1942 images belonging to 5 classes.
Found 484 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg16 = VGG16(input_shape = (100,100, 3), weights = 'imagenet', include_top = False)

In [ ]:
# Don't train the existing weights

for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
# Fully Connected Layers

flatten = Flatten()(vgg16.output)

dense = Dense(256, activation = 'relu')(flatten)
dense = Dropout(0.5)(dense)
dense = Dense(100, activation = 'relu')(dense)
dense = Dropout(0.3)(dense)

# Output Layer
prediction = Dense(5, activation = 'softmax')(dense)

In [ ]:
model = Model(inputs = vgg16.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0   

In [ ]:
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')

In [ ]:
# Fit the model
history = model.fit_generator(train_data,validation_data=valid_data,epochs=15)

<ipython-input-25-5c39e155f334>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_data,validation_data=valid_data,epochs=15)


Epoch 1/15
31/31 [==============================] - 333s 11s/step - loss: 0.4605 - accuracy: 0.8167 - val_loss: 0.0272 - val_accuracy: 0.9938
Epoch 2/15
31/31 [==============================] - 301s 10s/step - loss: 0.0533 - accuracy: 0.9835 - val_loss: 0.0087 - val_accuracy: 0.9959
Epoch 3/15
31/31 [==============================] - 301s 10s/step - loss: 0.0313 - accuracy: 0.9918 - val_loss: 0.0066 - val_accuracy: 0.9979
Epoch 4/15
31/31 [==============================] - 271s 9s/step - loss: 0.0293 - accuracy: 0.9912 - val_loss: 3.6731e-04 - val_accuracy: 1.0000
Epoch 5/15
31/31 [==============================] - 270s 9s/step - loss: 0.0127 - accuracy: 0.9954 - val_loss: 6.2042e-04 - val_accuracy: 1.0000
Epoch 6/15
31/31 [==============================] - 297s 10s/step - loss: 0.0134 - accuracy: 0.9974 - val_loss: 5.8751e-04 - val_accuracy: 1.0000
Epoch 7/15
31/31 [==============================] - 272s 9s/step - loss: 0.0081 - accuracy: 0.9990 - val_loss: 0.0056 - val_accuracy: 0.99

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# Load the test image
from google.colab import drive
drive.mount('/content/drive')
test_img_path = '/content/drive/MyDrive/fruit_classification/data/testimg.png'
img = image.load_img(test_img_path, target_size=(100, 100))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the index to the class label
class_labels = train_data.class_indices
inv_class_labels = {v: k for k, v in class_labels.items()}
predicted_class_label = inv_class_labels[predicted_class_index]

print("Predicted Fruit Class:", predicted_class_label)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 406ms/step
Predicted Fruit Class: Watermelon


In [ ]:
# Make predictions
predictions = model.predict(img_array)
predicted_class_index = tf.argmax(predictions[0]).numpy()

1/1 [==============================] - 0s 245ms/step


In [ ]:
# Get the predicted class name
predicted_class_name = inv_class_labels[predicted_class_index]

print("Predicted Fruit Class:", predicted_class_name)